## This notebook solves for the best coalition and compares Gurobi Solver against GCS-Q. Set distributions, sparsities, seeds, and agents in the 2nd code cell.
### You can set use_adv in the 3rd code cell to True and use_gr to False in order to only run only advantage, for normal or random distrubution above 50 agents

In [3]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import numpy as np
import pandas as pd
from importlib import reload
import min_cut_solvers
reload(min_cut_solvers)
import min_cut_solvers
import GCS
import utils
reload(utils)
from utils import *
import time
import matplotlib.pyplot as plt
import json

In [3]:
data = {'Agents' : [],
        'Sparsity' : [],
        'Distribution' : [],
        'Seed' : [],
        'Problem Edge Sum' : [],
        'Gurobi Value' : [],
        'Gurobi Solution' : [],
        'Gurobi TTE' : [],
        'Gurobi n.Calls' : [],
        'Advantage Value' : [],
        'Advantage Solution' : [],
        'Advantage Call Time' : [],
        'Advantage Access Time' : [],
        'Advantage TTE' : [],
        'TTE-CallTime+AcTime':[],
        'Advantage n.Calls' : []}

   
distributions = [   
    laplace,
    normal,
    random
]
sparsities = [0,0.5,1]
seeds = [121,122,123,124,125]
agents = np.arange(2,10)

use_adv = True
use_gr = True

In [ ]:
t_id = time.time()
file_name = './results/full'+str(t_id)

isg = generate_induced_subgraph_game(laplace, 10, sparsity=1)
label = "none"
no_print = GCS.gcs_calls(isg, min_cut_solver = min_cut_solvers.gurobi_solver, ext_label = label)

   
for n in agents:
    print(f'\nAgents: {n}',end='')
    for distribution in distributions:
        print("\n   ", distribution.__name__, end='')
        for sparsity in sparsities:
            print(f"\n      Sparsity:{sparsity}", end=" ")
            adv_sum = 0
            gr_sum = 0
            seed_start = time.time()
            for seed in seeds:
                print(f"[{seed}]", end=" ")

                np.random.seed(seed=seed)
                induced_subgraph_game = generate_induced_subgraph_game(distribution,n , sparsity=sparsity)
                pes = sum(induced_subgraph_game.values())
                label = 'ad_vs_gr_'+str(seed)+'_'+str(n)+'_'+distribution.__name__+'_'+str(sparsity)+'_'+str(t_id)
                
                # Advantage
                if use_adv:
                    print("Adv", end=" ")
                    start_time = time.time()
                    dwave_advantage_solution, dwave_advantage_value, adv_calls = GCS.gcs_calls(induced_subgraph_game, min_cut_solver = min_cut_solvers.min_cut_dwave_advantage, chain_str = 0.3, ext_label = label)
                    adv_tte = (time.time() - start_time)
                    path = f"runtime/{seed}/{n}/{distribution.__name__}/{sparsity}/{str(t_id)}/"
                    with open(path+'log.json', 'r') as f:
                        time_dic = json.load(f)
                    adv_at = round(time_dic['qpu_access_time']/(10**6),6)
                    adv_ct = time_dic['call_time']
                    adv_sum +=adv_tte
                else:
                    dwave_advantage_solution, dwave_advantage_value, adv_calls, adv_tte, adv_at, adv_ct = None, None, None, None, None, None
                

                # Gurobi
                if use_gr:
                    print("Gr", end='')      
                    start_time = time.time()
                    gurobi_solution , gurobi_value, gr_calls = GCS.gcs_calls(induced_subgraph_game, min_cut_solver = min_cut_solvers.gurobi_solver, ext_label = label)   
                    gr_tte = (time.time() - start_time)
                    gr_sum +=gr_tte
                else:    
                    gurobi_solution , gurobi_value, gr_calls, gr_tte = None, None, None, None
                    
                                
                
                data['Agents'].append(n)
                data['Sparsity'].append(sparsity)
                data['Distribution'].append(distribution.__name__)
                data['Seed'].append(seed)
                data['Problem Edge Sum'].append(pes)
                data['Advantage Value'].append(dwave_advantage_value)
                data['Advantage TTE'].append(adv_tte)
                data['Advantage Solution'].append(dwave_advantage_solution)
                data['Advantage Call Time'].append(adv_ct)
                data['Advantage Access Time'].append(adv_at)
                data['TTE-CallTime+AcTime'].append(adv_tte-adv_ct+adv_at)
                data['Advantage n.Calls'].append(adv_calls)
                data['Gurobi Value'].append(gurobi_value)
                data['Gurobi TTE'].append(gr_tte)
                data['Gurobi Solution'].append(gurobi_solution)
                data['Gurobi n.Calls'].append(gr_calls)

                data_pd = pd.DataFrame(data)
                data_pd.to_csv(file_name + '.csv', index=False)
            print(f"Seed Time : {round(time.time()-seed_start,2)}Adv Time: {round(adv_sum,2)}, Gr Time: {round(gr_sum,2)}")
                    
print(f"done!\n Filename:{file_name+'.csv'}, TTE: {round(time.time()-t_id,2)}")


Restricted license - for non-production use only - expires 2024-10-28

Agents: 17
    laplace
      Sparsity:0 [121] Adv Gr[122] Adv Gr[123] Adv Gr[124] Adv Gr[125] Adv GrSeed Time : 116.89Adv Time: 116.55, Gr Time: 0.31

      Sparsity:0.5 [121] Adv Gr[122] Adv Gr[123] Adv Gr[124] Adv Gr[125] Adv GrSeed Time : 209.52Adv Time: 209.22, Gr Time: 0.29

      Sparsity:1 [121] Adv Gr[122] Adv Gr[123] Adv Gr[124] Adv Gr[125] Adv GrSeed Time : 255.35Adv Time: 255.17, Gr Time: 0.13

    normal
      Sparsity:0 [121] Adv Gr[122] Adv Gr[123] Adv Gr[124] Adv Gr[125] Adv GrSeed Time : 162.0Adv Time: 161.66, Gr Time: 0.33

      Sparsity:0.5 [121] Adv Gr[122] Adv Gr[123] Adv Gr[124] Adv Gr[125] Adv GrSeed Time : 178.9Adv Time: 178.63, Gr Time: 0.24

      Sparsity:1 [121] Adv Gr[122] Adv Gr[123] Adv Gr[124] Adv Gr[125] Adv GrSeed Time : 240.14Adv Time: 239.95, Gr Time: 0.11

    random
      Sparsity:0 [121] Adv Gr[122] Adv Gr[123] Adv Gr[124] Adv Gr[125] Adv GrSeed Time : 168.04Adv Time: 167.64, G

In [12]:
df = pd.read_csv('./results/advonly_norm1692291062.1215425_99to109.csv')
df

,Agents,Sparsity,Distribution,Seed,Problem Edge Sum,Gurobi Value,Gurobi Solution,Gurobi TTE,Gurobi n.Calls,Advantage Value,Advantage Solution,Advantage Call Time,Advantage Access Time,Advantage TTE,TTE-CallTime+AcTime,Advantage n.Calls
0,99,0,normal,123,17776.81,NaN,NaN,NaN,NaN,14075.70,"['1,10,16,18,19,23,26,28,29,30,33,34,35,36,39,...",214.128175,0.383541,321.395905,107.651271,14
1,100,0,normal,123,18408.28,NaN,NaN,NaN,NaN,13551.30,"['28,67', '3,6,7,8,9,29,37,58,83,97', '4,25,36...",146.359373,0.375916,366.024089,220.040632,14
2,101,0,normal,123,18556.10,NaN,NaN,NaN,NaN,14105.39,"['2,3,4,5,6,7,10,11,12,15,16,21,22,23,26,27,29...",142.294256,0.365047,224.280528,82.351319,13
3,102,0,normal,123,18926.04,NaN,NaN,NaN,NaN,14319.88,"['10', '3', '1,5,6,7,8,12,15,16,17,18,19,21,22...",175.211692,0.288121,242.099264,67.175692,10
4,103,0,normal,123,19278.04,NaN,NaN,NaN,NaN,15137.12,"['74', '26', '1,17,22,28,54,66,68,77,84,85,88'...",126.013200,0.302572,200.668051,74.957423,11
5,104,0,normal,123,19198.02,NaN,NaN,NaN,NaN,16278.43,"['18,65', '32', '1,2,5,6,7,9,14,17,20,21,23,24...",149.872596,0.308142,224.249461,74.685007,11
6,105,0,normal,123,19786.80,NaN,NaN,NaN,NaN,16280.88,"['29,33,50,79,82,83', '2,4,5,6,12,14,16,17,21,...",192.842362,0.295835,262.641184,70.094657,10
7,106,0,normal,123,20303.90,NaN,NaN,NaN,NaN,14167.79,"['1,13,14,15,19,21,22,23,29,31,32,34,37,38,39,...",199.517134,0.397162,300.956439,101.836467,14
8,107,0,normal,123,20665.33,NaN,NaN,NaN,NaN,16665.19,"['38,56', '1,64,68,101', '2,5,6,11,15,17,19,20...",182.343655,0.449728,488.268110,306.374183,16
9,108,0,normal,123,20681.08,NaN,NaN,NaN,NaN,15299.34,"['1,3,6,7,12,13,15,19,20,21,23,28,31,34,35,36,...",428.567547,0.359889,525.289712,97.082054,13


In [6]:
df = pd.read_csv('./results/advonly_norm1692277764.3472605_88to99.csv')
df

,Agents,Sparsity,Distribution,Seed,Problem Edge Sum,Gurobi Value,Gurobi Solution,Gurobi TTE,Gurobi n.Calls,Advantage Value,Advantage Solution,Advantage Call Time,Advantage Access Time,Advantage TTE,TTE-CallTime+AcTime,Advantage n.Calls
0,88,0,normal,123,12331.62,NaN,NaN,NaN,NaN,10982.94,"['2,3,5,6,9,10,14,15,21,24,30,33,35,39,41,42,4...",39.325141,0.306579,111.556143,72.537581,11
1,89,0,normal,123,13068.65,NaN,NaN,NaN,NaN,13028.30,"['10,12,34,35,44,64,72', '70', '1,7,14,17,21,2...",45.762032,0.251252,144.692700,99.181920,9
2,90,0,normal,123,13699.15,NaN,NaN,NaN,NaN,12159.51,"['3,8,9,11,14,15,19,24,26,27,35,36,38,41,46,49...",103.721791,0.378211,182.953023,79.609443,13
3,91,0,normal,123,14365.75,NaN,NaN,NaN,NaN,13737.79,"['3,11,17,26,41,44,59,76,82', '12,15,56', '77'...",88.784612,0.306202,163.919374,75.440964,11
4,92,0,normal,123,14615.29,NaN,NaN,NaN,NaN,13481.08,"['2,4,5,6,7,10,11,16,18,21,26,31,32,33,34,36,3...",46.776249,0.281397,109.622663,63.127811,10
5,93,0,normal,123,15114.13,NaN,NaN,NaN,NaN,13159.40,"['6,10,22,28,36,40,62,84,86,87,90', '16', '1,4...",109.590995,0.314479,187.796372,78.519856,11
6,94,0,normal,123,15600.54,NaN,NaN,NaN,NaN,12821.05,"['7,16,17,20,24,29,30,33,36,41,42,43,48,49,53,...",131.340182,0.350758,266.648771,135.659347,13
7,95,0,normal,123,16128.09,NaN,NaN,NaN,NaN,15189.82,"['5,8,10,24,26,37,39,40,45,46,47,53,65,69,70,7...",87.665332,0.177194,124.889097,37.400960,6
8,96,0,normal,123,16571.96,NaN,NaN,NaN,NaN,12613.60,"['1,3,9,13,15,22,25,35,36,39,41,48,49,50,52,60...",80.310755,0.287545,145.909279,65.886069,10
9,97,0,normal,123,16860.81,NaN,NaN,NaN,NaN,13860.03,"['1,7,14,15,16,17,18,19,20,21,22,23,24,26,29,3...",117.372142,0.441106,209.300340,92.369304,15


In [3]:
df = pd.read_csv('./results/advonly1690458825.3038795_53to59.csv')
df

,Agents,Sparsity,Distribution,Seed,Problem Edge Sum,Gurobi Value,Gurobi Solution,Gurobi TTE,Gurobi n.Calls,Advantage Value,Advantage Solution,Advantage Call Time,Advantage Access Time,Advantage TTE,TTE-CallTime+AcTime,Advantage n.Calls
0,53,0.0,laplace,123,20289.34,NaN,NaN,NaN,NaN,20289.34,"['1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18...",5.303905,0.030840,17.373642,12.100577,1
1,53,0.0,laplace,123,20289.34,NaN,NaN,NaN,NaN,20289.34,"['1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18...",13.811562,0.031805,27.065491,13.285734,1
2,53,0.0,laplace,123,20289.34,NaN,NaN,NaN,NaN,20289.34,"['1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18...",5.851797,0.029382,12.249952,6.427537,1
3,53,0.5,laplace,123,12648.17,NaN,NaN,NaN,NaN,12648.17,"['1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18...",5.215475,0.029848,11.344907,6.159280,1
4,53,1.0,laplace,123,-1161.57,NaN,NaN,NaN,NaN,310.49,"['22,36,38', '46', '13', '12', '26', '23', '49...",0.829623,0.808979,232.683465,232.662821,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,58,0.5,random,123,1119.31,NaN,NaN,NaN,NaN,7593.38,"['23', '5,17,22,31,33,37,38,40,47,50,55,56', '...",50.178004,0.244799,128.938680,79.005475,9
61,58,1.0,random,123,-332.08,NaN,NaN,NaN,NaN,1012.00,"['2,9,10,19,41,43,51', '46', '48', '23', '29',...",1.330843,0.689248,227.584384,226.942789,28
62,59,0.0,random,123,-748.66,NaN,NaN,NaN,NaN,10054.72,"['1,2,7,8,18,21,23,24,26,40,46,58', '3,4,5,6,1...",44.767251,0.218385,115.220220,70.671353,8
63,59,0.5,random,123,1230.26,NaN,NaN,NaN,NaN,8363.01,"['6,8,10,25,28,29,32,33,34,35,39,43,44,48,52,5...",45.233709,0.181636,91.698699,46.646626,7


In [11]:
# longest TTE for advantage is for the following problem:
i = df['Advantage TTE'].idxmax()
print(df.loc[i])

Agents                                                                  94
Sparsity                                                                 0
Distribution                                                        normal
Seed                                                                   123
Problem Edge Sum                                                  15600.54
Gurobi Value                                                           NaN
Gurobi Solution                                                        NaN
Gurobi TTE                                                             NaN
Gurobi n.Calls                                                         NaN
Advantage Value                                                   12821.05
Advantage Solution       ['7,16,17,20,24,29,30,33,36,41,42,43,48,49,53,...
Advantage Call Time                                             131.340182
Advantage Access Time                                             0.350758
Advantage TTE            

## The next code cell demonstrates the sudden rise in execution time of Gurobi for normal and random distribution. With more than 50 agents the TTE becomes infeasible. We use the max TTE for advantage as boundary and would at most solve 100 agent problems, which are possible for laplace distribution.

In [ ]:
distributions = [   
    laplace,
    normal,
    random
]
for dist in distributions:
    print(dist.__name__)
    print(" ")
    for n in range(40,100):
        np.random.seed(seed=123)
        induced_subgraph_game = generate_induced_subgraph_game(dist,n ,0)
        start = time.time()
        gurobi_solution , gurobi_value, gr_calls = GCS.gcs_calls(induced_subgraph_game, min_cut_solver = min_cut_solvers.gurobi_solver)
        print(f"Agents: {n}, Time: {time.time()-start}")
        if time.time()-start >485:
            print("Time limit")
            break

laplace
 
Agents: 40, Time: 0.12036848068237305
Agents: 41, Time: 0.12102746963500977
Agents: 42, Time: 0.12513303756713867
Agents: 43, Time: 0.13903164863586426
Agents: 44, Time: 0.1360306739807129
Agents: 45, Time: 0.1470334529876709
Agents: 46, Time: 0.14876198768615723
Agents: 47, Time: 0.16103649139404297
Agents: 48, Time: 0.16303634643554688
Agents: 49, Time: 0.17328286170959473
Agents: 50, Time: 0.18104076385498047
Agents: 51, Time: 0.189042329788208
Agents: 52, Time: 0.19175148010253906
Agents: 53, Time: 0.20104551315307617
Agents: 54, Time: 0.2083284854888916
Agents: 55, Time: 0.2210536003112793
Agents: 56, Time: 0.22618818283081055
Agents: 57, Time: 0.236053466796875
Agents: 58, Time: 0.24319839477539062
Agents: 59, Time: 0.251056432723999
Agents: 60, Time: 0.25537633895874023
Agents: 61, Time: 0.26606035232543945
Agents: 62, Time: 0.28020215034484863
Agents: 63, Time: 0.2830791473388672
Agents: 64, Time: 0.30788278579711914
Agents: 65, Time: 0.30310678482055664
Agents: 66, T